# Extract phones from TextGrid

> Incomplete, needs work

- branch: master
- badges: false
- hidden: true
- categories: [hsi, textgrid]

The input is the output from MFA, so tiers have specific names.

In [1]:
from praatio import textgrid

def get_phone_sequences(tgfile):
    tg = textgrid.openTextgrid(tgfile, False)

    def get_tier_list(tiername):
        utterances = []
        tier = tg.getTier(tiername)
        for entry in tier.entries:
            text = entry.label.strip()
            if text == "":
                continue
            utterance = {}
            utterance["start"] = entry.start
            utterance["end"] = entry.end
            utterance["text"] = text
            utterances.append(utterance)
        return utterances
    
    utterances = get_tier_list("utterances")
    words = get_tier_list("words")
    phones = get_tier_list("phones")

    i = j = k = 0
    while i < len(utterances):
        utterance = utterances[i]
        utterance["words"] = []

        while j < len(words) and words[j]["end"] <= utterance["end"]:
            word = words[j]
            if word["end"] <= utterance["end"]:
                word["phones"] = []

                while k < len(phones) and phones[k]["end"] <= word["end"]:
                    phone = phones[k]
                    if phone["end"] <= word["end"]:
                        word["phones"].append(phone)
                    k += 1

                utterance["words"].append(word)
            j += 1
        i += 1

    return utterances

In [2]:
def get_dictionary(phone_sequence):
    lines = set()
    for utterance in phone_sequence:
        for word in utterance["words"]:
            phone_seq = [p["text"] for p in word["phones"]]
            lines.add(f"{word['text']}\t{' '.join(phone_seq)}")
    return list(lines)

In [3]:
MAPPING = """
AA0 ɑː
AA1 ˈɑː
AA2 ˌɑː
AE0 æ
AE1 ˈæ
AE2 ˌæ
AH0 ə
AH0 ɐ
AH1 ˈʌ
AH2 ˌʌ
AO0 ɔː
AO1 ˈɔː
AO2 ˌɔː
AW0 aʊ
AW1 ˈaʊ
AW2 ˌaʊ
AY0 aɪ
AY1 ˈaɪ
AY2 ˌaɪ
B b
CH tʃ
D d
DH ð
EH0 ɛ
EH1 ˈɛ
EH2 ˌɛ
ER0 ɚ
ER1 ˈɜː
ER2 ˌɜː
EY0 eɪ
EY1 ˈeɪ
EY2 ˌeɪ
F f
G ɡ
HH h
IH0 ɪ
IH1 ˈɪ
IH2 ˌɪ
IY0 i
IY1 ˈiː
IY2 ˌiː
JH dʒ
K k
L l
M m
N n
NG ŋ
OW0 oʊ
OW1 ˈoʊ
OW2 ˌoʊ
OY0 ɔɪ
OY1 ˈɔɪ
OY2 ˌɔɪ
P p
R ɹ
S s
SH ʃ
T t
TH θ
UH0 ʊ
UH1 ˈʊ
UH2 ˌʊ
UW0 uː
UW1 ˈuː
UW2 ˌuː
V v
W w
Y j
Z z
ZH ʒ
"""

In [4]:
extended = True
if extended:
    MAPPING += """
DX ɾ
"""

In [5]:
cmudict_to_espeak = {}
for line in MAPPING.split("\n"):
    if line == "":
        continue
    line = line.strip()
    parts = line.split(" ")

    if len(parts) != 2:
        print(line)
        continue
    k, v = line.split(" ")
    if not k in cmudict_to_espeak:
        cmudict_to_espeak[k] = v

In [6]:
def espeakify(phlist, sep=""):
    output = []
    if phlist == ["spn"] or phlist == ["sil"]:
        return ""
    for phone in phlist:
        if phone == "":
            continue
        if " " in phone:
            output += [cmudict_to_espeak[x] for x in phone.split(" ")]
        else:
            output.append(cmudict_to_espeak[phone])
    return sep.join(output)

In [7]:
import re

def get_utterances_like_espeak(phone_sequence):
    pairs = {}
    for utterance in phone_sequence:
        words = []
        text = utterance["text"]
        if text.startswith("[") and text.endswith("]"):
            continue
        for word in utterance["words"]:
            phones = [x["text"] for x in word["phones"]]
            phword = espeakify(phones)
            words.append(phword)
        joined = " ".join(words)
        joined = re.sub("  +", " ", joined)
        pairs[text] = joined
    return pairs

In [8]:
ps = get_phone_sequences("/Users/joregan/Playing/textgrids_shared/hsi_7_0719_209_001_main.TextGrid")
# ps = get_phone_sequences("/Users/joregan/Desktop/hsi_7_0719_227_003_inter.TextGrid")
utts = get_utterances_like_espeak(ps)

In [9]:
def run_espeak(text):
    phon = !echo "{text}"| espeak -v en-us --ipa -q
    return (" ".join(phon)).strip()

In [10]:
FIXES = {
    "kˈɑːlɚ": "kˈʌlɚ",
    "dʒɪs": "dʒˈʌs",
    "fɹɚ": "fɚ",
    "fɹə": "fɚ",
}

In [11]:
def cmudict_fixes(text):
    words = text.split(" ")
    out = []
    for word in words:
        if word in FIXES:
            out.append(FIXES[word])
        else:
            out.append(word)
    return " ".join(out)

In [12]:
def assimilations(text):
    text = text.replace("ð ð", " ð")
    text = text.replace("d ð", " ð")
    text = text.replace("d d", " d")
    return text

In [13]:
for utt in utts:
    print(utt)
    print(assimilations(cmudict_fixes(utts[utt])))
    print(run_espeak(utt))
    print()

Okay.
əkˈeɪ
oʊkˈeɪ

So hi, welcome.
sˈoʊ hˈaɪ wˈɛlkəm
sˌoʊ hˈaɪ  wˈɛlkʌm

Yeah, thank you very much.
jˈæ θˈæŋk jˈuː bˈɛɹi mˈʌtʃ
jˈɛh  θˈæŋk juː vˈɛɹi mˈʌtʃ

I really put a lot of effort to, you know, decorate it and ah put all the tiny details that I like ah on my space you know. I... it's really important for me, all the details that I... I put in each thing. For example, I mean, all the vases and the sculptures that I have, I mean I... just detailedly selected them.
ˈaɪ ɹˈiːli pˈʊt ə lˈɑːt əv ˈɛfɚt tˈuː jˈuː nˈoʊ dˈɛkɚˌeɪt ɪt ən ə pˈʊt ˈɑːl ðə tˈaɪni dˈiːteɪlz ðˈæt ˈaɪ lˈaɪk hˈæ ˈɑːn mˈaɪ spˈeɪs jˈuː nˈoʊ ˈaɪ ts ɹˈiːli ɪmpˈɔːɹtən fɹɛ mˈiː hˈɑːl ðə dˈiːteɪlz ðˈæt ˈaɪ ˈaɪ pˈʊt ˈɪn ˈiːtʃ θˈɪŋ fɚ ɪɡzˈæmpəl ˈaɪ mˈiːn ˈɑːl ðə vˈeɪsəz ən ðˈʌ skˈʌlptʃɚz ðˈæt ˈaɪ hˈæb ˈaɪ mˈiːn ˈaɪ dʒˈʌs dɪtˈeɪli səlˈɛktɪ ðˈɛm
aɪ ɹˈiəli pˌʊt ɐ lˈɑːt ʌv ˈɛfɚt tuː  juː nˈoʊ  dˈɛkɚɹˌeɪt ɪt ænd ˈɑː pˌʊt ˈɔːl ðə tˈaɪni diːtˈeɪlz ðæt aɪ lˈaɪk ˈɑː ˌɑːn maɪ spˈeɪs juː nˈoʊ  ˈaɪ  ɪts ɹˈiəli ɪmpˈoːɹtənt fɔːɹ mˌiː  ˈɔːl

In [14]:
filestem = "hsi_7_0719_209_003_main"
audio_dir = "/Users/joregan/Playing/hsi/audio/"

with open("/tmp/run_ffmpeg.sh", "w") as outf:
    for item in ps:
        start = item["start"]
        dur = item["end"] - item["start"]
        outf.write(f"ffmpeg -i {audio_dir}{filestem}.wav -acodec pcm_s16le -ac 1 -ar 16000 -ss {start} -t {dur} /tmp/phoninput/{filestem}_{start}_{item['end']}.wav\n")